###### Imports and Settings

In [50]:
import pandas as pd
import numpy as np
import requests
from collections import deque
from functools import reduce
import pickle
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("..") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
shorttnplaces = geodict.shorttnplaces
import sqlite3 as sq
#functions
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

In [51]:
#this already has the 2020 PL number for 2020 population
data = pd.read_csv('../../Data Downloads/CensusBureau_HistoricalPopulation.csv')
data = data[['NAME', '1900','1910', '1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010', '2020']]
data = data.set_index(['NAME'])
#data.columns = pd.MultiIndex.from_product([['Population'], data.columns, ['None']])
data.head()

,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020
NAME,,,,,,,,,,,,,
"Adams city, Tennessee",0,542,672,0,0,0,0,458,600,587,566,633,624
"Ashland City town, Tennessee",0,641,649,712,957,1024,1400,2027,2329,2552,3641,4541,5193
"Belle Meade city, Tennessee",0,0,0,0,2061,2831,3082,0,3182,2839,2943,2912,2901
"Berry Hill city, Tennessee",0,0,0,0,0,1248,1551,0,1113,802,674,537,2112
"Brentwood city, Tennessee",0,0,0,0,0,0,0,0,9431,16392,23445,37060,45373


In [52]:
data = data.transpose()
GNRCCounties = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                data['Rutherford County, Tennessee']]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                   data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                   data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                   data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                   data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                   data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                   data['Rutherford County, Tennessee'],data['Maury County, Tennessee']]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
               data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
               data['Williamson County, Tennessee'],data['Rutherford County, Tennessee'],
               data['Maury County, Tennessee']]
data['MPO'] = sum(MPOCounties)
RuthInc = [data['Eagleville city, Tennessee'],data['La Vergne city, Tennessee'],
           data['Murfreesboro city, Tennessee'],data['Smyrna town, Tennessee']]
data['Rutherford Incorporated'] = sum(RuthInc)
data['Rutherford Unincorporated'] = data['Rutherford County, Tennessee'] - data['Rutherford Incorporated']
WilsonInc = [data['Lebanon city, Tennessee'],data['Mount Juliet city, Tennessee'],
             data['Watertown city, Tennessee']]
data['Wilson Incorporated'] = sum(WilsonInc)
data['Wilson Unincorporated'] = data['Wilson County, Tennessee'] - data['Wilson Incorporated']
CheathInc = [data['Ashland City town, Tennessee'],data['Kingston Springs town, Tennessee'],
             data['Pegram town, Tennessee'],data['Pleasant View city, Tennessee']]
data['Cheatham Incorporated'] = sum(CheathInc)
data['Cheatham Unincorporated'] = data['Cheatham County, Tennessee'] - data['Cheatham Incorporated']
DicksInc = [data['Burns town, Tennessee'],data['Charlotte town, Tennessee'],
            data['Dickson city, Tennessee'],data['Slayden town, Tennessee'],
            data['Vanleer town, Tennessee'],data['White Bluff town, Tennessee']]
data['Dickson Incorporated'] = sum(DicksInc)
data['Dickson Unincorporated'] = data['Dickson County, Tennessee'] - data['Dickson Incorporated']
HumphInc = [data['McEwen city, Tennessee'],data['New Johnsonville city, Tennessee'],
            data['Waverly city, Tennessee']]
data['Humphreys Incorporated'] = sum(HumphInc)
data['Humphreys Unincorporated'] = data['Humphreys County, Tennessee'] - data['Humphreys Incorporated']
data['Montgomery Incorporated'] = data['Clarksville city, Tennessee']
data['Montgomery Unincorporated'] = data['Montgomery County, Tennessee'] - data['Montgomery Incorporated']

In [53]:
data = data.transpose()

In [54]:
data.head()

,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020
NAME,,,,,,,,,,,,,
"Adams city, Tennessee",0,542,672,0,0,0,0,458,600,587,566,633,624
"Ashland City town, Tennessee",0,641,649,712,957,1024,1400,2027,2329,2552,3641,4541,5193
"Belle Meade city, Tennessee",0,0,0,0,2061,2831,3082,0,3182,2839,2943,2912,2901
"Berry Hill city, Tennessee",0,0,0,0,0,1248,1551,0,1113,802,674,537,2112
"Brentwood city, Tennessee",0,0,0,0,0,0,0,0,9431,16392,23445,37060,45373


In [55]:
cols = data.columns
data = data.reset_index(drop = False)
data = data.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Population')#, values = cols)

In [56]:
data['Year'] = data['Year'].astype(int)

In [57]:
histpop = data

In [58]:
histpop['GEO_ID'] = histpop['NAME'].map(geotogeoid)
histpop['Source'] = 'Historical Decennial Censuses'

In [59]:
histpop.head()

,NAME,Year,Population,GEO_ID,Source
0,"Adams city, Tennessee",1900,0,1600000US4700200,Historical Decennial Censuses
1,"Ashland City town, Tennessee",1900,0,1600000US4702180,Historical Decennial Censuses
2,"Belle Meade city, Tennessee",1900,0,1600000US4704620,Historical Decennial Censuses
3,"Berry Hill city, Tennessee",1900,0,1600000US4705140,Historical Decennial Censuses
4,"Brentwood city, Tennessee",1900,0,1600000US4708280,Historical Decennial Censuses


In [60]:
histpop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NAME        1118 non-null   object
 1   Year        1118 non-null   int32 
 2   Population  1118 non-null   int64 
 3   GEO_ID      884 non-null    object
 4   Source      1118 non-null   object
dtypes: int32(1), int64(1), object(3)
memory usage: 39.4+ KB


In [61]:
#export to feather to join to all tidydata before creating change metrics
histpop.to_feather('../../Outputs/HIST_PopulationTIDY')

In [62]:
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [63]:
data.head()

,NAME,Year,Population,GEO_ID,Source
0,"Adams city, Tennessee",1900,0,1600000US4700200,Historical Decennial Censuses
1,"Ashland City town, Tennessee",1900,0,1600000US4702180,Historical Decennial Censuses
2,"Belle Meade city, Tennessee",1900,0,1600000US4704620,Historical Decennial Censuses
3,"Berry Hill city, Tennessee",1900,0,1600000US4705140,Historical Decennial Censuses
4,"Brentwood city, Tennessee",1900,0,1600000US4708280,Historical Decennial Censuses


In [64]:
data = data.drop(columns = ['GEO_ID', 'Source'])
data.head()

,NAME,Year,Population
0,"Adams city, Tennessee",1900,0
1,"Ashland City town, Tennessee",1900,0
2,"Belle Meade city, Tennessee",1900,0
3,"Berry Hill city, Tennessee",1900,0
4,"Brentwood city, Tennessee",1900,0


In [65]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Population                                                                    
Year                               1900 1910 1920 1930 1940  1950  1960  1970  1980  1990  2000  2010  2020
NAME                                                                                                       
Adams city, Tennessee                 0  542  672    0    0     0     0   458   600   587   566   633   624
Ashland City town, Tennessee          0  641  649  712  957  1024  1400  2027  2329  2552  3641  4541  5193

In [66]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Population                                                                     
                                   1900 1910 1920 1930  1940  1950  1960  1970  1980  1990  2000  2010  2020
                                   None None None None  None  None  None  None  None  None  None  None  None
NAME                                                                                                        
Adams city, Tennessee                 0  542  672    0     0     0     0   458   600   587   566   633   624
Ashland City town, Tennessee          0  641  649  712   957  1024  1400  2027  2329  2552  3641  4541  5193
Belle Meade city, Tennessee           0    0    0    0  2061  2831  3082     0  3182  2839  2943  2912  2901

In [67]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [68]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [69]:
data.head()

Population                                                                          Population % Change  \
                                   1900 1910 1920 1930  1940  1950  1960  1970  1980   1990   2000   2010   2020                None   
                                   None None None None  None  None  None  None  None   None   None   None   None           1900-1910   
NAME                                                                                                                                   
Adams city, Tennessee                 0  542  672    0     0     0     0   458   600    587    566    633    624                 inf   
Ashland City town, Tennessee          0  641  649  712   957  1024  1400  2027  2329   2552   3641   4541   5193                 inf   
Belle Meade city, Tennessee           0    0    0    0  2061  2831  3082     0  3182   2839   2943   2912   2901                 NaN   
Berry Hill city, Tennessee            0    0    0    0     0  1248  1551     0  1113    802    674    537   2112                 NaN   
Brentwood city, Tennessee             0    0    0    0     0     0     0     0  9431  16392  23445  37060  45373                 NaN   

                             Population Change Population % Change Population Change Population % Change Population Change Population % Change  \
                                          None                None              None                None              None                None   
                                     1900-1910           1900-1920         1900-1920           1900-1930         1900-1930           1900-1940   
NAME                                                                                                                                             
Adams city, Tennessee                      542                 inf               672                 NaN                 0                 NaN   
Ashland City town, Tennessee               641                 inf               649                 inf               712                 inf   
Belle Meade city, Tennessee                  0                 NaN                 0                 NaN                 0                 inf   
Berry Hill city, Tennessee                   0                 NaN                 0                 NaN                 0                 NaN   
Brentwood city, Tennessee                    0                 NaN                 0                 NaN                 0                 NaN   

                             Population Change Population % Change Population Change Population % Change Population Change Population % Change  \
                                          None                None              None                None              None                None   
                                     1900-1940           1900-1950         1900-1950           1900-1960         1900-1960           1900-1970   
NAME                                                                                                                                             
Adams city, Tennessee                        0                 NaN                 0                 NaN                 0                 inf   
Ashland City town, Tennessee               957                 inf              1024                 inf              1400                 inf   
Belle Meade city, Tennessee               2061                 inf              2831                 inf              3082                 NaN   
Berry Hill city, Tennessee                   0                 inf              1248                 inf              1551                 NaN   
Brentwood city, Tennessee                    0                 NaN                 0                 NaN                 0                 NaN   

                             Population Change Population % Change Population Change Population % Change Population Change Population % Change  \
                            

In [70]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [71]:
data.head()

,NAME,Year,Time Frame,Population,Population % Change,Population Change
0,"Adams city, Tennessee",1900,None,0.0,NaN,NaN
1,"Adams city, Tennessee",1910,None,542.0,NaN,NaN
2,"Adams city, Tennessee",1920,None,672.0,NaN,NaN
3,"Adams city, Tennessee",1930,None,0.0,NaN,NaN
4,"Adams city, Tennessee",1940,None,0.0,NaN,NaN


In [72]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Historical Decennial Censuses'

In [73]:
#final check
data.head()

,NAME,Year,Time Frame,Population,Population % Change,Population Change,GEO_ID,Source
0,"Adams city, Tennessee",1900,None,0.0,NaN,NaN,1600000US4700200,Historical Decennial Censuses
1,"Adams city, Tennessee",1910,None,542.0,NaN,NaN,1600000US4700200,Historical Decennial Censuses
2,"Adams city, Tennessee",1920,None,672.0,NaN,NaN,1600000US4700200,Historical Decennial Censuses
3,"Adams city, Tennessee",1930,None,0.0,NaN,NaN,1600000US4700200,Historical Decennial Censuses
4,"Adams city, Tennessee",1940,None,0.0,NaN,NaN,1600000US4700200,Historical Decennial Censuses


In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NAME                 7826 non-null   object 
 1   Year                 7826 non-null   object 
 2   Time Frame           7826 non-null   object 
 3   Population           1118 non-null   float64
 4   Population % Change  6056 non-null   float64
 5   Population Change    6708 non-null   float64
 6   GEO_ID               6188 non-null   object 
 7   Source               7826 non-null   object 
dtypes: float64(3), object(5)
memory usage: 489.2+ KB


In [74]:
#export to the SQLite database
conn = sq.connect('../../Outputs/CensusBureau.db')
data.to_sql('CensusBureau_HistoricalPopulation_Annual_Change', conn, if_exists = 'replace', index = False)

7826